# Data-X Spring 2019: Homework 7

### Webscraping



In this homework, you will do some exercises with web-scraping.

## Name: Zihui Xu

## SID: 3034305174



### Fun with Webscraping & Text manipulation


## 1. Statistics in Presidential Debates

Your first task is to scrape Presidential Debates from the Commission of Presidential Debates website: https://www.debates.org/voter-education/debate-transcripts/

To do this, you are not allowed to manually look up the URLs that you need, instead you have to scrape them. The root url to be scraped is the one listed above, namely: https://www.debates.org/voter-education/debate-transcripts/


1. By using `requests` and `BeautifulSoup` find all the links / URLs on the website that links to transcriptions of **First Presidential Debates** from the years [1988, 1984, 1976, 1960]. In total you should find 4 links / URLs that fulfill this criteria. **Print the urls.**

2. When you have a list of the URLs your task is to create a Data Frame with some statistics (see example of output below):
    1. Scrape the title of each link and use that as the column name in your Data Frame. 
    2. Count how long the transcript of the debate is (as in the number of characters in transcription string). Feel free to include `\` characters in your count, but remove any breakline characters, i.e. `\n`. You will get credit if your count is +/- 10% from our result.
    3. Count how many times the word **war** was used in the different debates. Note that you have to convert the text in a smart way (to not count the word **warranty** for example, but counting **war.**, **war!**, **war,** or **War** etc.
    4. Also scrape the most common used word in the debate, and write how many times it was used. Note that you have to use the same strategy as in C in order to do this.
    
    **Print your final output result.**
    
**Tips:**

___

In order to solve the questions above, it can be useful to work with Regular Expressions and explore methods on strings like `.strip(), .replace(), .find(), .count(), .lower()` etc. Both are very powerful tools to do string processing in Python. To count common words for example I used a `Counter` object and a Regular expression pattern for only words, see example:

```python
    from collections import Counter
    import re

    counts = Counter(re.findall(r"[\w']+", text.lower()))
```

Read more about Regular Expressions here: https://docs.python.org/3/howto/regex.html
    
    
**Example output of all of the answers to Question 1.2:**


![pres_stats_2](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/presidents_stats_2.jpg)



----

.




In [1]:
import requests # HTTP library for getting and posting content etc.
import bs4 as bs # We can query markup languages for specific content
from collections import Counter
import re
import math
import pandas as pd

source=requests.get('https://www.debates.org/voter-education/debate-transcripts')
soup= bs.BeautifulSoup(source.content, features='html.parser') 

year=['1988', '1984', '1976', '1960']
links = soup.find_all('a')

url=[]

for l in links:
    if "First" in l.text:
        link=l.get('href')
        print(link)
        for i in year:
            if i in link:
                url.append(link)

/voter-education/debate-transcripts/october-15-1992-first-half-debate-transcript/
/voter-education/debate-transcripts/october-11-1992-first-half-debate-transcript/
/voter-education/debate-transcripts/september-26-2016-debate-transcript/
/voter-education/debate-transcripts/october-3-2012-debate-transcript/
/voter-education/debate-transcripts/2008-debate-transcript/
/voter-education/debate-transcripts/september-30-2004-debate-transcript/
/voter-education/debate-transcripts/october-3-2000-transcript/
/voter-education/debate-transcripts/october-6-1996-debate-transcript/
/voter-education/debate-transcripts/october-11-1992-first-half-debate-transcript/
/voter-education/debate-transcripts/october-15-1992-first-half-debate-transcript/
/voter-education/debate-transcripts/september-25-1988-debate-transcript/
/voter-education/debate-transcripts/october-7-1984-debate-transcript/
/voter-education/debate-transcripts/september-23-1976-debate-transcript/
/voter-education/debate-transcripts/september-2

In [2]:
title_all=[]
total=[]
sum_1=[]
most_word=[]
most_num=[]

for i in url:
    source_new=requests.get('https://www.debates.org{}'.format(i))
    soup_new= bs.BeautifulSoup(source_new.content, features='html.parser')
# get the title for the dataframe 
    title_1=soup_new.find('strong').text
    title_2=soup_new.find_all('p')[1].text
    title=title_1+ ': '+ title_2
    title_all.append(title)
# count characters in the transcript
    phrase = re.findall(r"[a-zA-Z\-]+", soup_new.text.lower())
    str = ' '.join(phrase)
    total.append(len(str))
# count how many time "war" shows up 
    counts = phrase.count("wars")+phrase.count('war')+phrase.count('war.')
    sum_1.append(counts)
# most common used word in the debate
    most =Counter(phrase).most_common(1)
    most_word.append(most[0][0])
# write how many times it was used
    most_num.append(most[0][1])
   

In [3]:
df=pd.DataFrame([total,sum_1,most_word,most_num],columns=title_all,index=['Debate char length','War_count','Most_common_w','Most_common_w_count'])
df

,"September 25, 1988: The First Bush-Dukakis Presidential Debate","October 7, 1984: The First Reagan-Mondale Presidential Debate","September 23, 1976: The First Carter-Ford Presidential Debate","September 26, 1960: The First Kennedy-Nixon Presidential Debate"
Debate char length,87115,85262,79578,61162
War_count,11,3,7,3
Most_common_w,the,the,the,the
Most_common_w_count,802,868,857,780


    
## 2. Download and read in specific line from many data sets

Scrape the first 27 data sets from this URL http://people.sc.fsu.edu/~jburkardt/datasets/regression/ (i.e.`x01.txt` - `x27.txt`). Then, save the 5th line in each data set, this should be the name of the data set author (get rid of the `#` symbol, the white spaces and the comma at the end). 

Count how many times (with a Python function) each author is the reference for one of the 27 data sets. Showcase your results, sorted, with the most common author name first and how many times he appeared in data sets. Use a Pandas DataFrame to show your results, see example. **Print your final output result.**

**Example output of the answer for Question 2:**

![author_stats](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/data_authors.png)


In [4]:
source=requests.get('http://people.sc.fsu.edu/~jburkardt/datasets/regression/')
soup= bs.BeautifulSoup(source.content, features='html.parser') 

links=soup.find_all('a')
url=[]
for l in links:
    if "x" in l.text:
        link=l.get('href')
        source_new='http://people.sc.fsu.edu/~jburkardt/datasets/regression/{}'.format(link)
        url.append(source_new)

In [5]:
fifth=[]
for i in url[0:27]:
    source_new=requests.get(i)
    soup_new= bs.BeautifulSoup(source_new.content, features='html.parser')
    #rint(soup_new)
    for line in soup_new:
        content=line.split('\n')
        fifth.append((content[4]).strip('#').strip(' ').strip(','))
word_fre=Counter(fifth).items()

freq=[]
name=[]
for x in word_fre:
        freq.append([x][0][1])
        name.append([x][0][0])
        
    



In [6]:
df=pd.DataFrame([name,freq]).transpose()
df.columns=['Authors','Counts']
df_final=df.sort_values(by='Counts',ascending=False)
df_final.set_index('Authors')

,Counts
Authors,
Helmut Spaeth,16
"S Chatterjee, B Price",3
R J Freund and P D Minton,2
D G Kleinbaum and L L Kupper,2
"S C Narula, J F Wellington",2
K A Brownlee,1
S Chatterjee and B Price,1
